In [4]:
import sys
import graphviz
import numpy as np
import uuid

from sklearn.model_selection import train_test_split

sys.path.append('..')
from Decision_tree.decision_Tree import DecisionTree

In [23]:
data = np.load('../Data/fashion_train.npy')
X = data[:,:-1]
y = data[:,-1]
print("X shape: ", X.shape)
print("y shape: ", y.shape)

data = X > 15

clf = DecisionTree(data, y, debug = False, max_depth = 5)
clf.fit()

print("Train score", clf.score(X, y))

X shape:  (10000, 784)
y shape:  (10000,)


Train score 0.6865


In [24]:
labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Shirt']
label_colors = ['#0000ff', '#ffac1c', '#228b22', '#ee4b2b', '#800080']


In [25]:
def readTree(parent, current_node, dot):

    i = uuid.uuid1()
    if current_node.data["leaf"]:
        dot.node(str(i), 
                 labels[current_node.data["class"]],
                 style='filled',
                 fontcolor='white',
                 color=label_colors[current_node.data["class"]])
        dot.edge(str(parent), str(i))
        return

    dot.node(str(i),
             "Feature: " + str(current_node.data["feature"])
             + " <= " + str(current_node.data["cutoff"][0])
             + "\n" + "Datapoints: " + str(current_node.data["count_datapoints"])
             + "\n" + "Impurity: " + str(current_node.data["impurity"]),
             color='black',
             style='filled',
             fillcolor='beige',
             shape='box')
    dot.edge(str(parent), str(i))
    
    readTree(i, current_node.leftChild, dot)
    readTree(i, current_node.rightChild, dot)


In [26]:
dot = graphviz.Digraph(comment='Decision Tree')
i = uuid.uuid1()

tree = clf.tree
dot.node(str(i),
         "Feature: " + "feature: " + str(tree.data["feature"])
        + " <= " + str(tree.data["cutoff"][0])
        + "\n" + "Datapoints: " + str(tree.data["count_datapoints"])
        + "\n" + "Impurity: " + str(tree.data["impurity"]),
        color='black',
        style='filled',
        fillcolor='beige',
        shape='box')

readTree(i, tree.leftChild, dot)
readTree(i, tree.rightChild, dot)


In [28]:
dot.render('tree.gv', view=True, format='png').replace('\\', '/')

'tree.gv.png'